In [1]:
from urllib import *
import json
import regex as re
import PySimpleGUI as sg
import webbrowser
import sys
import pprint
from dataclasses import dataclass
import hyperlink
import requests

In [2]:
host       = "localhost"
port       = "8983"
collection = "restaurant"
qt         = "select"
url        = 'http://' + host + ':' + port + '/solr/' + collection + '/' + qt + '?'

q          = "q=*%3A*"
#fl         = "fl=id,name"
#fq         = "fq="
#rows       = "rows=10"
wt         = "wt=json"

params     = [ q, wt ] 
p          = "&".join(params)

In [3]:
connection = request.urlopen(url+p)

if wt == "wt=json":
  response = json.load(connection) 
else:
  response = eval(connection.read())

print ("Number of hits: " + str(response['response']['numFound']))
for i in range(len(response['response']['docs'])):
    print(response['response']['docs'][i])
#pprint.pprint(response['response']['docs'][0]['webpage'])
#print(len(response['response']['docs']))

Number of hits: 1013
{'name': ['1. Positano @ RP'], 'type': ['Italian', ' European'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d21180746-Reviews-Positano_RP-Singapore.html'], 'id': '118c8d31-feaa-4986-a1f9-89c921fb25fa', '_version_': 1694013575789543424}
{'name': ['2. Grand Shanghai Restaurant'], 'type': ['Chinese', ' Asian'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1145149-Reviews-Grand_Shanghai_Restaurant-Singapore.html'], 'id': '1d0bb2f9-9dd8-4732-8736-b25bd2a63f30', '_version_': 1694013575897546752}
{'name': ['3. Fu Lin Men (NSRCC)'], 'type': ['Chinese', ' Seafood'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d17171783-Reviews-Fu_Lin_Men_NSRCC-Singapore.html'], 'id': 'ebd438d8-029f-483d-9d4b-93964d76e2fe', '_version_': 1694013575900692480}
{'name': ['4. Entre-Nous creperie'], 'type': ['French', ' European'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1193730-Reviews-En

In [4]:
layout = [[sg.Text('Search'),sg.InputText(), sg.Button('Search', bind_return_key=True, key="_SEARCH_")],
          [sg.Button("Quit", key = '_QUIT_')],
          [sg.Listbox([], font = ["Arial", 16],size=(1000,40),enable_events=True, key = '_listbox_')]
          
          ]
#[sg.Output(size=(1400,650), key = '_output_')]
# Create the window http://localhost:8888/notebooks/Documents/School%20Work/2021%20Yr%203%20Sem%202/CE%204034/info-retrieval/Webpage%20demo.ipynb#
window = sg.Window("Demo Search Engine", layout, size=(1440, 850))

#I hardcoded such that it will only search for values in 'type' field, 
#need to update to make it search for all fields

#Also need to make the hyperlinks clickable


# Create an event loop
while True:
    event, values = window.read()
    
    # End program if user closes window or
    # presses the OK button
    if event == "_QUIT_" or event == sg.WIN_CLOSED:
        break
    if event == '_SEARCH_':
        
        wt         = "wt=json"
        
        string = str(values[0])
        
        x = string.split()
        print(x)
        
        lay1 = []
        lay2 = []
        lay = []
        
        for terms in x:
            connection = requests.get('http://localhost:8983/solr/restaurant/select', params={"q": "type"+":" + terms, 'wt' : 'json', 'rows' : 100,  'indent' : True})
            print(connection.json())
            print("")
            response=connection.json()

        
            for i in range(len(response['response']['docs'])):
        
                lay1.append(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['type'])
                #p = response['response']['docs'][i]['webpage'][0]
                #lay1.append(p)
            #print(p)
            
            #print(lay1)
            #print("")
            
            connection = requests.get('http://localhost:8983/solr/restaurant/select', params={"q": "name"+":" + terms, 'wt' : 'json', 'rows' : 100, 'indent' : True})
        #print(connection.json())
            response=connection.json()
        
            for i in range(len(response['response']['docs'])):
        
                lay2.append(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['name'])
            #print(response['response']['docs'][i]['type'])
                #p = response['response']['docs'][i]['webpage'][0]
                #lay2.append(p)
            #print(p)
            
            #print(lay2)
            #print("")
            
        lay = lay1+lay2
        #print(lay)

        window.Element('_listbox_').Update(values=lay)
    if event == '_listbox_':

        restaurant_name = values['_listbox_'][0][0]
        r = re.findall('\s(.*)',restaurant_name)
        
        connection = requests.get('http://localhost:8983/solr/comment/select', params={"q": "restaurant"+":"+r[0], 'wt' : 'json', 'indent' : True})
        response=connection.json()
        
        lay = []
        for i in range(len(response['response']['docs'])):
            lay.append(response['response']['docs'][i]['comment'])
            #print(response['response']['docs'][i]['comment'])
        window.Element('_listbox_').Update(values=lay)
        #print(r[0])

window.close()

* Error performing wm_overrideredirect *
['asian', 'pizza', 'kick']
{'responseHeader': {'status': 0, 'QTime': 0, 'params': {'q': 'type:asian', 'indent': 'True', 'rows': '100', 'wt': 'json'}}, 'response': {'numFound': 310, 'start': 0, 'numFoundExact': True, 'docs': [{'name': ['266. Spize'], 'type': ['Asian'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d9877078-Reviews-Spize-Singapore.html'], 'id': '06282913-fd96-42a2-a976-aa152e22a33a', '_version_': 1694013576192196608}, {'name': ['299. Lim Chee Guan'], 'type': ['Asian'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d9740103-Reviews-Lim_Chee_Guan-Singapore.html'], 'id': 'b9f30341-1fd5-4ca5-a621-0d2b3dd73f2a', '_version_': 1694013576208973826}, {'name': ['445. Whole Earth'], 'type': ['Asian'], 'webpage': ['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1746501-Reviews-Whole_Earth-Singapore.html'], 'id': '4357622b-711b-40b7-b937-50d4fba95dab', '_version_': 1694013576280276993}

['hello']
{'responseHeader': {'status': 0, 'QTime': 0, 'params': {'q': 'type:hello', 'indent': 'True', 'rows': '100', 'wt': 'json'}}, 'response': {'numFound': 0, 'start': 0, 'numFoundExact': True, 'docs': []}}

